In [1]:
import torch
import numpy as np
import normflows as nf
import FCYeast_simulator

from matplotlib import pyplot as plt
from eZplot import FCBSCD_plot
from tqdm import tqdm

import gc


enable_cuda = True
CUDA_LAUNCH_BLOCKING=1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

FCYeast_simulator.adjust_device(device)

model_file = 'FCYeast_network.pt'

In [2]:
# Define target
target = FCYeast_simulator.target()
context_size = 6

In [3]:
# Define flows
latent_size = 1
hidden_units = 128
hidden_layers = 5

flows = [nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 64,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 128,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 256,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size),      
         nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, 512,num_context_channels=context_size,tail_bound=30),
         nf.flows.LULinearPermute(latent_size)
         ]


# Set base distribution
q0 = nf.distributions.DiagGaussian(1)#, trainable=False)
    
# Construct flow model
model = nf.ConditionalNormalizingFlow(q0, flows, target)

# Move model on GPU if available
model = model.to(device)

In [4]:
try:
    model.load_state_dict(torch.load(model_file))
    #model.eval()  # Set the model to evaluation mode
    print('loading pretrained network')
except:
    print('starting from scratch')

starting from scratch


In [5]:
max_iter   = 20000
max_iter   = 4000
show_iter  = 100
n_batch    = 16

x = target.sample(n=1024*1024)
batch_size = x.size(0)//n_batch

x,context = x[:,0].reshape(-1,1)*1.0,x[:,1:]

batches = torch.arange(x.size(0)).reshape(n_batch,-1)

In [6]:
# Train model
loss_hist = np.array([])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)

In [7]:
for it in tqdm(range(max_iter)):
    loss_epoch = np.array([])

    for it2 in (range(n_batch)):  
        optimizer.zero_grad() 
        gc.collect()
        
        # Compute loss
        batch = batches[it2]
        loss = -model.log_prob(x[batch], context[batch]).mean()
        
        # Do backprop and optimizer step
        if ~(torch.isnan(loss) | torch.isinf(loss)):
            loss.backward()
            optimizer.step()
        
        loss_epoch = np.append(loss_epoch, loss.to('cpu').item())

    # Log loss
    loss_hist = np.append(loss_hist, np.mean(loss_epoch))

    if (it+1)%show_iter==0:
        index = (loss_hist.size+np.arange(-int(2.5*show_iter),0))
        FCBSCD_plot(model,target,loss_hist,index[index>=0],figs_direc = 'FCYeast_network_perform')
        #print('yoo')

    if (it + 1) % (3*show_iter) == 0:
        with torch.no_grad():
            torch.save(model.state_dict(), model_file)

    with torch.no_grad():
        samples_new = target.sample(n=64*1024)

        x[:samples_new.size(0)] = samples_new[:,0].reshape((-1,1))
        context[:samples_new.size(0)] = samples_new[:,1:]

        shuffle_index = torch.randperm(x.size(0))
        x = (x[shuffle_index]).contiguous()
        context = (context[shuffle_index]).contiguous()

  8%|▊         | 333/4000 [29:32<5:25:13,  5.32s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 7.79 GiB total capacity; 6.02 GiB already allocated; 60.44 MiB free; 6.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.save(model.state_dict(), model_file)

In [10]:
x.size()

torch.Size([1048576, 1])